In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Set random seed for reproducibility
torch.manual_seed(42)

In [3]:
# Load the data
data = pd.read_csv('Bird_Data_missing.txt', delimiter=' ')
incomplete_records = data.iloc[148:, [3, 4]]
data = data.iloc[:148, :]

In [4]:
# Extract features and labels
X = data.loc[:, ['Wing', 'Weight']]
y = data.loc[:, ['Firewing', 'Shadowhawk', 'Thunderbeaks']]

In [6]:
# Split the data into training, validation, and testing sets
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, test_size=0.5, random_state=42)

In [7]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [8]:
# Create a custom dataset
class BirdDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [9]:
# Create dataloaders
train_dataset = BirdDataset(X_train, y_train)
val_dataset = BirdDataset(X_val, y_val)
test_dataset = BirdDataset(X_test, y_test)

In [10]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [11]:
# Define the neural network model
class BirdClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BirdClassifier, self).__init__()
        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)
        return x

In [12]:
# Set up hyperparameter grid
param_grid = {
    'hidden_dim': [8, 12, 16, 24, 32],
    'learning_rate': [0.1, 0.01, 0.001],
    'weight_decay': [0.1, 0.01, 0.001, 0.0001, 0]
}

In [13]:
# Initialize variables to track the best model
best_accuracy = 0.0
best_params = None
best_model = None

In [14]:
# Iterate over the hyperparameter grid
for hidden_dim in param_grid['hidden_dim']:
    for learning_rate in param_grid['learning_rate']:
        for weight_decay in param_grid['weight_decay']:
            print(f"Training model with {hidden_dim} hidden units, learning rate {learning_rate}, and weight decay {weight_decay}")

            model = BirdClassifier(input_dim=2, hidden_dim=hidden_dim, output_dim=3)
            criterion = nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

            # Training loop
            num_epochs = 100
            for epoch in range(num_epochs):
                model.train()
                for X_batch, y_batch in train_loader:
                    optimizer.zero_grad()
                    y_pred = model(X_batch)
                    loss = criterion(y_pred, y_batch)
                    loss.backward()
                    optimizer.step()

            # Evaluation on validation set
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for X_val, y_val in val_loader:
                    y_pred = model(X_val)
                    _, predicted = torch.max(y_pred.data, 1)
                    total += y_val.size(0)
                    correct += (predicted == torch.max(y_val, 1)[1]).sum().item()

            accuracy = correct / total
            print(f"Accuracy on validation set: {accuracy * 100:.2f}%")

            # Update the best model if the current one is better
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'hidden_dim': hidden_dim, 'learning_rate': learning_rate, 'weight_decay': weight_decay}
                best_model = model

Training model with 8 hidden units, learning rate 0.1, and weight decay 0.1
Accuracy on validation set: 81.82%
Training model with 8 hidden units, learning rate 0.1, and weight decay 0.01
Accuracy on validation set: 95.45%
Training model with 8 hidden units, learning rate 0.1, and weight decay 0.001
Accuracy on validation set: 95.45%
Training model with 8 hidden units, learning rate 0.1, and weight decay 0.0001
Accuracy on validation set: 90.91%
Training model with 8 hidden units, learning rate 0.1, and weight decay 0
Accuracy on validation set: 90.91%
Training model with 8 hidden units, learning rate 0.01, and weight decay 0.1
Accuracy on validation set: 63.64%
Training model with 8 hidden units, learning rate 0.01, and weight decay 0.01
Accuracy on validation set: 90.91%
Training model with 8 hidden units, learning rate 0.01, and weight decay 0.001
Accuracy on validation set: 95.45%
Training model with 8 hidden units, learning rate 0.01, and weight decay 0.0001
Accuracy on validation

In [15]:
# Output details of the optimal model
print(f"Best Model Results \nHidden Units: {best_params['hidden_dim']} \nLearning Rate: {best_params['learning_rate']} \nWeight Decay: {best_params['weight_decay']} \nAccuracy on validation set: {best_accuracy * 100:.2f}%")

Best Model Results 
Hidden Units: 8 
Learning Rate: 0.1 
Weight Decay: 0.01 
Accuracy on validation set: 95.45%


In [16]:
# Evaluate the best model on the test set
best_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_test, y_test in test_loader:
        y_pred = best_model(X_test)
        _, predicted = torch.max(y_pred.data, 1)
        total += y_test.size(0)
        correct += (predicted == torch.max(y_test, 1)[1]).sum().item()

In [17]:
accuracy_test = correct / total
print(f"Accuracy of the best model on the test set: {accuracy_test * 100:.2f}%")

Accuracy of the best model on the test set: 95.65%
